<a href="https://colab.research.google.com/github/sjmin99/2019.Summer.Intern/blob/master/%EB%B0%B0%EC%B6%94kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import torch
import torch.nn.functional as F
import numpy as np
import torch.optim as optim 

In [6]:
!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle -v
!kaggle competitions download -c regression-cabbage-price
!unzip regression-cabbage-price.zip

-rw-r--r-- 1 root root 63 May  3 05:12 kaggle.json
Found existing installation: kaggle 1.5.6
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.1)
Processing /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01/kaggle-1.5.6-py3-none-any.whl
Kaggle API 1.5.6
  0% 0.00/29.7k [00:00<?, ?B/s]
100% 29.7k/29.7k [00:00<00:00, 25.2MB/s]
Archive:  regression-cabbage-price.zip
  inflating: submit_sample.csv       
  inflating: test_cabbage_price.csv  
  inflating: train_cabbage_price.csv  


In [0]:
torch.manual_seed(1)

pd_data = pd.read_csv('train_cabbage_price.csv')
x_train = torch.FloatTensor(np.array(pd_data.iloc[:,1:5]))
y_train = torch.FloatTensor(np.array(pd_data.iloc[:,5]))



In [43]:
W = torch.zeros((4,1), requires_grad= True)
b = torch.zeros(1, requires_grad= True)
epochs = 1000
lr = 1e-4

optimizer = optim.SGD((W,b), lr = lr, momentum=0.9, nesterov= True)

for epoch in range(epochs+1):
  # import pdb;pdb.set_trace()
  H = x_train.matmul(W)+b
  # cost = ((H-y_train) ** 2).sum(dim=0).mean()
  cost = torch.mean((H-y_train)**2)


  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    print("Epoch : {:4d}/{}, Cost : {:.6f}".format(
        epoch, epochs, cost.item()))
    

Epoch :    0/1000, Cost : 12610920.000000
Epoch :  100/1000, Cost : 2456089.250000
Epoch :  200/1000, Cost : 2441934.250000
Epoch :  300/1000, Cost : 2428242.000000
Epoch :  400/1000, Cost : 2414934.750000
Epoch :  500/1000, Cost : 2401985.250000
Epoch :  600/1000, Cost : 2389397.500000
Epoch :  700/1000, Cost : 2377137.500000
Epoch :  800/1000, Cost : 2365210.750000
Epoch :  900/1000, Cost : 2353608.750000
Epoch : 1000/1000, Cost : 2342313.250000


In [57]:
pd_test = pd.read_csv('test_cabbage_price.csv')

0    13.5
1     1.1
2    12.0
3    16.6
4    22.4
5    18.8
Name: avgTemp, dtype: float64

In [0]:
x_test = torch.FloatTensor(np.array(pd_test.iloc[:,1:5]))
H = x_test.matmul(W)+b

Id = np.array([i for i in range(len(x_test))]).reshape(-1,1)

Expected = H.detach().numpy().reshape(-1,1)
result = np.hstack([Id,Expected])

df = pd.DataFrame(result, columns=["Id","Expected"])
df = df.astype(int)
# df.to_csv("result.csv", index=False, header= True)
df.to_csv("result.csv", index=False, header= True)

In [103]:
!kaggle competitions submit -c regression-cabbage-price -f result.csv -m "Assignment"

100% 54.0/54.0 [00:00<00:00, 243B/s]
Successfully submitted to Regression : Cabbage price